In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd

In [ ]:
url = ''
response = requests()

In [20]:
starbucks = pd.read_csv('directory.csv')
starbucks = starbucks[['Brand','City','State/Province', 'Postcode']]
starbucks.columns = ['brand', 'city', 'state', 'zip']
starbucks = starbucks[starbucks['city']=='Los Angeles']
starbucks

,brand,city,state,zip
13681,Starbucks,Los Angeles,CA,90038
13682,Teavana,Los Angeles,CA,90064
13683,Starbucks,Los Angeles,CA,90045
13684,Starbucks,Los Angeles,CA,900274306
13685,Starbucks,Los Angeles,CA,900242811
13686,Starbucks,Los Angeles,CA,90045
13687,Starbucks,Los Angeles,CA,90014
13688,Starbucks,Los Angeles,CA,900287509
13689,Starbucks,Los Angeles,CA,90045
13690,Starbucks,Los Angeles,CA,900641719
